In [68]:
pip install asn1crypto

Note: you may need to restart the kernel to use updated packages.


In [69]:
pip install attrs


Note: you may need to restart the kernel to use updated packages.


In [70]:
pip install Automat


Note: you may need to restart the kernel to use updated packages.


In [71]:
pip install certifi


Note: you may need to restart the kernel to use updated packages.


In [72]:
pip install cffi


Note: you may need to restart the kernel to use updated packages.


In [73]:
pip install chardet


Note: you may need to restart the kernel to use updated packages.


In [74]:
pip install click


Note: you may need to restart the kernel to use updated packages.


In [75]:
pip install constantly

Note: you may need to restart the kernel to use updated packages.


In [76]:
pip install cryptography


Note: you may need to restart the kernel to use updated packages.


In [77]:
pip install cssselect


Note: you may need to restart the kernel to use updated packages.


In [78]:
pip install Flask


Note: you may need to restart the kernel to use updated packages.


In [79]:
pip install hyperlink


Note: you may need to restart the kernel to use updated packages.


In [80]:
pip install idna


Note: you may need to restart the kernel to use updated packages.


In [81]:
pip install incremental


Note: you may need to restart the kernel to use updated packages.


In [82]:
pip install itsdangerous


Note: you may need to restart the kernel to use updated packages.


In [83]:
pip install Jinja2


Note: you may need to restart the kernel to use updated packages.


In [84]:
pip install lxml


Note: you may need to restart the kernel to use updated packages.


In [85]:
pip install MarkupSafe


Note: you may need to restart the kernel to use updated packages.


In [86]:
pip install parsel


Note: you may need to restart the kernel to use updated packages.


In [87]:
pip install pyasn1


Note: you may need to restart the kernel to use updated packages.


In [88]:
pip install pyasn1 modules


Note: you may need to restart the kernel to use updated packages.


In [89]:
pip install pycparser


Note: you may need to restart the kernel to use updated packages.


In [90]:
pip install PyDispatcher


Note: you may need to restart the kernel to use updated packages.


In [91]:
pip install pymongo


Note: you may need to restart the kernel to use updated packages.


In [92]:
pip install pyOpenSSL


Note: you may need to restart the kernel to use updated packages.


In [93]:
pip install pywin32

ERROR: Could not find a version that satisfies the requirement pywin32 (from versions: none)
ERROR: No matching distribution found for pywin32
Note: you may need to restart the kernel to use updated packages.


In [94]:
pip install queuelib


Note: you may need to restart the kernel to use updated packages.


In [95]:
pip install requests


Note: you may need to restart the kernel to use updated packages.


In [96]:
pip install Scrapy


Note: you may need to restart the kernel to use updated packages.


In [97]:
pip install service identity


  Using cached service-0.6.0.tar.gz (12 kB)
ERROR: Could not find a version that satisfies the requirement identity (from versions: none)
ERROR: No matching distribution found for identity
Note: you may need to restart the kernel to use updated packages.


In [98]:
pip install six


Note: you may need to restart the kernel to use updated packages.


In [99]:
pip install Twisted


Note: you may need to restart the kernel to use updated packages.


In [100]:
pip install urllib3


Note: you may need to restart the kernel to use updated packages.


In [101]:
pip install w3lib


Note: you may need to restart the kernel to use updated packages.


In [102]:
pip install Werkzeug


Note: you may need to restart the kernel to use updated packages.


In [103]:
pip install wincertstore


Note: you may need to restart the kernel to use updated packages.


In [104]:
pip install zope.interface

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import scrapy


class ArticleItem(scrapy.Item):
    title = scrapy.Field()
    description = scrapy.Field()
    section = scrapy.Field()
    text = scrapy.Field()
    author = scrapy.Field()
    url = scrapy.Field()
    keywords = scrapy.Field()

In [ ]:
from scrapy import signals


class NewspySpiderMiddleware(object):
    @classmethod
    def from_crawler(cls, crawler):
        s = cls()
        crawler.signals.connect(s.spider_opened, signal=signals.spider_opened)
        return s
    def process_spider_input(self, response, spider):
        return None
    def process_spider_output(self, response, result, spider):
        for i in result:
            yield i
            
    def process_spider_exception(self, response, exception, spider):
        pass
    
    def process_start_requests(self, start_requests, spider):
        for r in start_requests:
            yield r
    def spider_opened(self, spider):
        spider.logger.info('Spider opened: %s' % spider.name)
        

In [ ]:
from pymongo import MongoClient
from scrapy import settings
import ssl


class NewspyPipeline(object):
    def process_item(self, item, spider):
        return item


class MongoDBPipeline(object):
    def __init__(self):
        connection = MongoClient(settings['MONGODB_HOST'],
                                 port=settings['MONGODB_PORT'],
                                 username=settings['MONGODB_USER'],
                                 password=settings['MONGODB_PASS'],
                                 ssl=True,
                                 ssl_cert_reqs=ssl.CERT_NONE)
        db = connection[settings['MONGODB_DB']]
        self.collection = db[settings['MONGODB_COLLECTION']]

    def process_item(self, item, spider):
        if item['text'] is not None:
            self.collection.insert(dict(item))
            print("Item wrote to MongoDB database %s/%s" %
                   (settings['MONGODB_DB'], settings['MONGODB_COLLECTION'])
            )
        return item
    

In [ ]:
BOT_NAME = 'newscrawler'
SPIDER_MODULES = ['newscrawler.spiders']
NEWSPIDER_MODULE = 'newscrawler.spiders'
ROBOTSTXT_OBEY = True
ITEM_PIPELINES = {
    'newscrawler.pipelines.MongoDBPipeline': 100
}

In [ ]:
import scrapy

from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor
import re


class NewsSpider(CrawlSpider):
    name = 'news'
    allowed_domains = ['bbc.com']
    start_urls = ['http://www.bbc.com/news']

    rules = (
        Rule(
            LinkExtractor(allow=(), unique=True),
            callback='parse_item'
        ),
    )

    def parse_item(self, response):
        page_type = response.xpath("//meta[@property='og:type']/@content").extract_first()

        if page_type == 'article':
            item = ArticleItem()
            item['url'] = response.url
            item['title'] = response.xpath("//meta[@property='og:title']/@content").extract_first() or \
                            response.xpath("//h1[@itemprop='headline']/text()").extract_first()
            item['description'] = response.xpath("//meta[@property='og:description']/@content").extract_first()
            item['author'] = response.xpath("//meta[@property='article:author']/@content").extract_first()
            item['section'] = response.xpath("//meta[@property='article:section']/@content").extract_first()
            item['keywords'] = response.xpath("//meta[@name='keywords']/@content").extract_first()

            text_contents = response.xpath("//div[@itemprop='articleBody' or @property='articleBody']/p").extract()
            item['text'] = re.sub(r'<.*?>', ' ', ''.join(text_contents)).strip() if text_contents else None

            return item